In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

train_data = pd.read_csv('/share/dutta/eyao/dataset/kaggle/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('/share/dutta/eyao/dataset/kaggle/house-prices-advanced-regression-techniques/test.csv')

X = train_data.drop(['Id', 'SalePrice'], axis=1)
y = np.log(train_data['SalePrice'])

test_ids = test_data['Id']
X_test = test_data.drop(['Id'], axis=1)

numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ])

xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
gb_model = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.05, random_state=42)

xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb_model)
])

rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', rf_model)
])

gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', gb_model)
])

param_grid = {
    'model__n_estimators': [500, 1000, 2000],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__max_depth': [3, 5, 7]
}

grid_search = GridSearchCV(xgb_pipeline, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X, y)

best_xgb_model = grid_search.best_estimator_

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

best_xgb_model.fit(X_train, y_train)
y_preds_xgb = best_xgb_model.predict(X_valid)
rmse_xgb = np.sqrt(mean_squared_error(y_valid, y_preds_xgb))
print(f'XGBoost RMSE: {rmse_xgb}')

rf_pipeline.fit(X_train, y_train)
y_preds_rf = rf_pipeline.predict(X_valid)
rmse_rf = np.sqrt(mean_squared_error(y_valid, y_preds_rf))
print(f'Random Forest RMSE: {rmse_rf}')

gb_pipeline.fit(X_train, y_train)
y_preds_gb = gb_pipeline.predict(X_valid)
rmse_gb = np.sqrt(mean_squared_error(y_valid, y_preds_gb))
print(f'Gradient Boosting RMSE: {rmse_gb}')

y_preds_ensemble = (y_preds_xgb + y_preds_rf + y_preds_gb) / 3
rmse_ensemble = np.sqrt(mean_squared_error(y_valid, y_preds_ensemble))
print(f'Ensemble RMSE: {rmse_ensemble}')

best_xgb_model.fit(X, y)
rf_pipeline.fit(X, y)
gb_pipeline.fit(X, y)

test_preds_xgb = np.exp(best_xgb_model.predict(X_test))
test_preds_rf = np.exp(rf_pipeline.predict(X_test))
test_preds_gb = np.exp(gb_pipeline.predict(X_test))
test_preds_ensemble = (test_preds_xgb + test_preds_rf + test_preds_gb) / 3

submission = pd.DataFrame({'Id': test_ids, 'SalePrice': test_preds_ensemble})
submission.to_csv('submission4.csv', index=False)